### Duality

The diet problem is a type of mixing problem in which we are given a set of foods and are asked to find a minimum-cost combination of these foods that satisfies a set of nutrient requirements. We considered a small instance of the diet problem in which we were given five types of food and were required to construct a diet that provided at least 21 units of iron and 12 units of calcium. Data on the foods were as follows.

<table>
<caption>Units of nutrients and cost per ounce</caption>
<tr>
<th> Food type </th> <th> Iron </th> <th> Calcium </th> <th> Cost </th>
</tr>
<tr>
<th> 1 </th> <th> 2 </th> <th> 0 </th> <th> 20 </th>
</tr>
<tr>
<th> 2 </th> <th> 0 </th> <th> 1 </th> <th> 10 </th>
</tr>
<tr>
<th> 3 </th> <th> 3 </th> <th> 2 </th> <th> 31 </th>
</tr>
<tr>
<th> 4 </th> <th> 1 </th> <th> 2 </th> <th> 11 </th>
</tr>
<tr>
<th> 5 </th> <th> 2 </th> <th> 1 </th> <th> 12 </th>
</tr>

We formulated this instance as
\begin{eqnarray}
\min_x && z = 20 x_1 + 10 x_2 + 31 x_3 + 11 x_4 + 12 x_5 \\
\mbox{s.t.} && 2 x_ 1 + 3 x_3 + x_4 + 2 x_5 \ge 21 \\
&& x_2 + 2 x_3 + 2 x_4 + x_5 \ge 12 \\
&& x_i \ge 0,\;\;i = 1,\ldots,5
\end{eqnarray}
where $x_i$ is the number of ounces of food type $i$ to include in our diet. Gurobi returned an optimal diet consisting of 1 unit of food type 4 and 10 units of food type 5, which satisfies the nutrient requirements at a cost of 131.

Sensitivity analysis revealed that the marginal cost of each nutrient is $4 \frac{1}{3}$ for iron and $3 \frac{1}{3}$ for calcium. These values came from the Pi attribute of the Constr object and are known as dual values or shadow prices. According to those prices, food types 1-3 (which were not used in the optimal diet) were all overpriced.

These dual values are instrumental in proving optimality, so let's explore how they are computed by formulating a complementary linear program called the dual. To do this, we'll take the perspective of a person who is selling iron and calcium pills to the dieter. The pill-seller must decide how much to charge per pill. Each pill contains one unit of its respective nutrient. And, the pills must be priced competitively with respect to the five food types available to the dieter.

Suppose the pill-seller charges $\pi_i$ for an iron pill and $\pi_c$ for a calcium pill. Since the dieter would need 21 iron pills and 12 calcium pills to meet his nutritional requirements, the pill-seller stands to collect a revenue of $21 \pi_i + 12 \pi_c$ if he prices the pills fairly. We enforce fair pricing by writing a structural constraint for each of the 5 food types. For example, food type 3 is equivalent to 3 iron pills and 2 calcium pills, and is priced at 31 per ounce, so $3 \pi_i + 2 \pi_c \le 31$.

<b> Diet Problem Dual </b>
<ul> 
<li> Decision Variables: 
<ul type="square">
<li>$\pi_i$ = price to charge per pill $i =$ iron, calcium</li>
</ul>
<li> Objective Function: (Maximize revenue)
<ul type="square">
<li> min $v = 21 \pi_i + 12 \pi_c$ </li>
</ul>
</li>
<li> Structural Constraints: </li>
<ul type="square">
<li> $2 \pi_i + 0 \pi_c \le 20$ (food 1) </li>
<li> $0 \pi_i + 1 \pi_c \le 10$ (food 2) </li>
<li> $3 \pi_i + 2 \pi_c \le 31$ (food 3) </li>
<li> $1 \pi_i + 2 \pi_c \le 11$ (food 4) </li>
<li> $2 \pi_i + 1 \pi_c \le 12$ (food 5) </li>

</ul>
<li> Nonnegativity constraints: (no negative prices)
<ul type="square">
<li> $\pi_i, \pi_c \ge 0$ </li>
</li>
</ul>

<b>Exercise: Formulate and solve the dual of the diet problem with Gurobi. </b>

<b>a. How is the maximum revenue for the pill-seller related to the minimum cost of the dieter? </b>

<b>b. What are the optimal pill prices? </b>

<b>c. What do the dual values of the dual LP represent? </b>

<b>d. What do the slacks on the constraints represent?</b>

In [1]:
# Create a gurobipy.Model object. 
# Gurobi will minimize by default. Change this by setting the ModelSense
# attribute to GRB.MAXIMIZE.
from gurobipy import *
m = Model()
m.ModelSense = GRB.MAXIMIZE

In [2]:
# Instantiate gurobipy.Var objects
# for each of our price decision variables.
pi_i = m.addVar(obj=21, name='price.iron')
pi_c = m.addVar(obj=12, name='price.calcium')

In [3]:
# Register the decision variables with the model via the update method.
m.update()

In [4]:
# Instantiate gurobipy.Constr objects to enforce competitive
# pricing with respect to all five food types
food1_con = m.addConstr(2*pi_i <= 20, 'food.1')
food2_con = m.addConstr(pi_c <= 10, 'food.2')
food3_con = m.addConstr(3*pi_i + 2*pi_c <= 31, 'food.3')
food4_con = m.addConstr(pi_i + 2*pi_c <= 11, 'food.4')
food5_con = m.addConstr(2*pi_i + pi_c <= 12, 'food.5')

In [5]:
# Optimize
m.update()
m.optimize()

Optimize a model with 5 rows, 2 columns and 8 nonzeros
Coefficient statistics:
  Matrix range    [1e+00, 3e+00]
  Objective range [1e+01, 2e+01]
  Bounds range    [0e+00, 0e+00]
  RHS range       [1e+01, 3e+01]
Presolve removed 3 rows and 0 columns
Presolve time: 0.01s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.9222500e+02   5.770250e+00   0.000000e+00      0s
       2    1.3100000e+02   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds
Optimal objective  1.310000000e+02


In [6]:
# Iterate over the decision variables to get the optimal prices
for var in m.getVars():
    print var.VarName, var.X

price.iron 4.33333333333
price.calcium 3.33333333333


In [7]:
# Iterate over the structural constraints and 
# print Pi and Slack for each
for constr in m.getConstrs():
    print constr.ConstrName, constr.Pi, constr.Slack

food.1 0.0 11.3333333333
food.2 0.0 6.66666666667
food.3 0.0 11.3333333333
food.4 1.0 0.0
food.5 10.0 0.0


### Reduced Costs

Let's officially establish a connection between the dual constraints and the concept of reduced costs. The dual constraints can be rewritten as:
\begin{eqnarray}
20 - 2 \pi_i &\ge& 0 \\
10 - \pi_c &\ge& 0 \\
31 - 3 \pi_i - 2 \pi_c &\ge& 0 \\
11 - \pi_i - 2 \pi_c &\ge& 0 \\
12 - 2 \pi_i - \pi_c &\ge& 0.
\end{eqnarray}
Consider the left-hand side of any of the above constraints. The constant represents the per ounce cost of one of the five food types. From that constant we subtract the value of the nutrients contained in one ounce of the food type (in terms of the nutrient prices $\pi_i$ and $\pi_c$. This difference is exactly the reduced cost of the original diet problem. Recall that the optimality conditions for the original diet LP were that the reduced costs had to be non-negative. That is exactly the condition the dual constraints enforce.

### Why do we care about reduced costs?

The dual values have a nice economic interpretation, and the good news is you don't actually have to formulate and solve the dual LP to find them. Gurobi automatically computes the dual values when it solves any LP, and you can access them via the Pi attribute of the Constr object.

Thinking in terms of the dual LP can answer some interesting questions about the three food types that had positive reduced costs (and therefore were not included in the optimal diet).

<b>Exercise: Suppose food type 3 (formerly priced at 31 per ounce) were put on sale for 20 per ounce. Assuming the shadow prices of iron and calcium remained the same, what would be the new reduced cost of food type 3? Would we include food type 3 in our diet? </b>

In [8]:
# The reduced cost of food type 3 is its per ounce cost (now 20) minus the value of the nutrients it provides
# Hint: You can access the shadow prices of iron and calcium via pi_i.X and pi_c.X, respectively.
20 - 3*pi_i.X - 2*pi_c.X

0.33333333333333304

In [9]:
# It looks like food type 3 is still overpriced. Let's verify this.
food3_con.RHS = 20
m.optimize()

Optimize a model with 5 rows, 2 columns and 8 nonzeros
Coefficient statistics:
  Matrix range    [1e+00, 3e+00]
  Objective range [1e+01, 2e+01]
  Bounds range    [0e+00, 0e+00]
  RHS range       [1e+01, 2e+01]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.3100000e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  1.310000000e+02


In [10]:
for constr in m.getConstrs():
    print constr.ConstrName, constr.Pi

food.1 0.0
food.2 0.0
food.3 0.0
food.4 1.0
food.5 10.0


<b> Exercise: How low does the per ounce cost of food type 3 need to go before the dieter would consider putting it in his diet? </b>

At a cost of 20 per ounce, food type 3 is still overpriced by $\frac{1}{3}$ per ounce. At a price of $19 \frac{2}{3}$ per ounce, the dieter would be indifferent between including and not including food type 3. At a price of, say 19 per ounce, the reduced cost of food type 3 would become negative and the dieter could actually lower his cost by including it.

In [11]:
food3_con.RHS = 19
m.optimize()

Optimize a model with 5 rows, 2 columns and 8 nonzeros
Coefficient statistics:
  Matrix range    [1e+00, 3e+00]
  Objective range [1e+01, 2e+01]
  Bounds range    [0e+00, 0e+00]
  RHS range       [1e+01, 2e+01]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.3100000e+02   1.666667e-01   0.000000e+00      0s
       1    1.2900000e+02   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.00 seconds
Optimal objective  1.290000000e+02


In [12]:
for constr in m.getConstrs():
    print constr.ConstrName, constr.Pi

food.1 0.0
food.2 0.0
food.3 3.0
food.4 0.0
food.5 6.0


<b> Exercise: Say food type 3 goes back to costing 31 per ounce, but now a new food type goes on sale that costs 20 per ounce and provides 2 units of iron and 3 of calcium. Would the dieter add this new food type to his diet?

In [13]:
# Change the cost of food type 3 back to 31 per ounce
food3_con.RHS = 31
m.optimize()

Optimize a model with 5 rows, 2 columns and 8 nonzeros
Coefficient statistics:
  Matrix range    [1e+00, 3e+00]
  Objective range [1e+01, 2e+01]
  Bounds range    [0e+00, 0e+00]
  RHS range       [1e+01, 3e+01]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.6500000e+02   4.000000e+01   0.000000e+00      0s
       1    1.3100000e+02   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds
Optimal objective  1.310000000e+02


In [14]:
# compute the reduced cost of the new food type
20 - 2*pi_i.X - 3*pi_c.X

1.333333333333334